In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import norm
from scipy import stats
import sqlite3 # needed to connect to database

### Importing the Dataset

In [2]:
#Connect to database
conn = sqlite3.connect('bmarket.db')

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables) #Only one table- we will call it df.
df = pd.read_sql("SELECT * FROM bank_marketing", conn)
df

             name
0  bank_marketing


,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status
0,32885,57 years,technician,married,high.school,no,no,yes,Cell,1,999,no
1,3170,55 years,unknown,married,unknown,unknown,yes,no,telephone,2,999,no
2,32207,33 years,blue-collar,married,basic.9y,no,no,no,cellular,1,999,no
3,9404,36 years,admin.,married,high.school,no,no,no,Telephone,4,999,no
4,14021,27 years,housemaid,married,high.school,no,None,no,Cell,2,999,no
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,6266,58 years,retired,married,professional.course,unknown,no,no,Telephone,2,999,no
41184,11285,37 years,management,married,university.degree,no,no,no,telephone,1,999,no
41185,38159,35 years,admin.,married,high.school,no,None,no,cellular,1,4,yes
41186,861,40 years,management,married,university.degree,no,None,no,telephone,2,999,no


## Initial Data Exploration:

Let's see if there are any duplicates first.

In [3]:
# Check for duplicated rows
duplicate_rows = df.duplicated()

# Count the number of duplicated rows
num_duplicate_rows = duplicate_rows.sum()

print(f"Number of duplicated rows: {num_duplicate_rows}")

# Display the duplicated rows (optional)
if num_duplicate_rows > 0:
  print("Duplicated rows:")
df[duplicate_rows]

Number of duplicated rows: 0


,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status


In [4]:
# Check for duplicates in 'Client ID' column
duplicate_client_ids = df.duplicated(subset=['Client ID'])

# Count the number of duplicate Client IDs
num_duplicate_client_ids = duplicate_client_ids.sum()

print(f"Number of duplicate Client IDs: {num_duplicate_client_ids}")

# Display the rows with duplicate Client IDs (optional)
if num_duplicate_client_ids > 0:
  print("Rows with duplicate Client IDs:")
df[duplicate_client_ids]

Number of duplicate Client IDs: 0


,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status


There were no duplicates in the dataset.

In [5]:
# Check for missing values
missing_values = df.isnull().sum()

print("Missing values in each column:")
print(missing_values)

# Display columns with missing values (optional)
columns_with_missing = missing_values[missing_values > 0]
if not columns_with_missing.empty:
  print("\nColumns with missing values:")
columns_with_missing

Missing values in each column:
Client ID                    0
Age                          0
Occupation                   0
Marital Status               0
Education Level              0
Credit Default               0
Housing Loan             24789
Personal Loan             4146
Contact Method               0
Campaign Calls               0
Previous Contact Days        0
Subscription Status          0
dtype: int64

Columns with missing values:


Housing Loan     24789
Personal Loan     4146
dtype: int64

There are some missing values in "Housing Loan" and "Personal Loan". Let's take a look at those columns.

In [6]:
# Get value counts for columns with missing values in the rows with missing data
rows_with_missing = df[df.isnull().any(axis=1)]
print("\nValue counts for columns with missing data in rows with missing values:")
for col in columns_with_missing.index:
  print(f"\nColumn: {col}")
  print(rows_with_missing[col].value_counts(dropna=False)) # include NaN count


Value counts for columns with missing data in rows with missing values:

Column: Housing Loan
Housing Loan
None       24789
yes          853
no           778
unknown       47
Name: count, dtype: int64

Column: Personal Loan
Personal Loan
no         18405
None        4146
yes         3385
unknown      531
Name: count, dtype: int64


The 'none' data is considered missing. We will assume that it means that no loan was taken, and that they will be filled with 'no' later.

Let's take a look at the other columns to see if we need to do anything to them.

In [7]:
df["Occupation"].value_counts()

Occupation
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

In [8]:
df["Marital Status"].value_counts()

Marital Status
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

In [9]:
df["Education Level"].value_counts()

Education Level
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

In [10]:
df["Credit Default"].value_counts()

Credit Default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

In [11]:
df["Contact Method"].value_counts()

Contact Method
Cell         13100
cellular     13044
Telephone     7585
telephone     7459
Name: count, dtype: int64

Cell and cellular are the same thing, and Telephone and telephone are the same thing.

In [12]:
df["Campaign Calls"].value_counts()

Campaign Calls
 1     15874
 2      9446
 3      4807
 4      2405
-1      1768
       ...  
 56        1
-41        1
 39        1
 37        1
-25        1
Name: count, Length: 70, dtype: int64

In [13]:
# Display the minimum and maximum values of the 'Campaign Calls' column
min_campaign_calls = df['Campaign Calls'].min()
max_campaign_calls = df['Campaign Calls'].max()

print(f"Minimum Campaign Calls: {min_campaign_calls}")
print(f"Maximum Campaign Calls: {max_campaign_calls}")

Minimum Campaign Calls: -41
Maximum Campaign Calls: 56


In [14]:
print(df['Campaign Calls'].describe())
print(df[df['Campaign Calls'] < 0]['Campaign Calls'].value_counts())
print(df[df['Campaign Calls'] == 0])
print(df[df['Campaign Calls'] >= 0]['Campaign Calls'].value_counts())

count    41188.000000
mean         2.051374
std          3.171345
min        -41.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         56.000000
Name: Campaign Calls, dtype: float64
Campaign Calls
-1     1768
-2     1124
-3      534
-4      246
-5      148
-6       86
-7       63
-8       35
-9       30
-11      21
-10      19
-12      18
-14      11
-13       8
-17       7
-23       5
-16       5
-15       4
-22       3
-20       3
-18       3
-19       2
-28       2
-35       2
-21       2
-29       1
-32       1
-41       1
-25       1
Name: count, dtype: int64
Empty DataFrame
Columns: [Client ID, Age, Occupation, Marital Status, Education Level, Credit Default, Housing Loan, Personal Loan, Contact Method, Campaign Calls, Previous Contact Days, Subscription Status]
Index: []
Campaign Calls
1     15874
2      9446
3      4807
4      2405
5      1451
6       893
7       566
8       365
9       253
10      206
11      156
12      107
13       84
14       

Campaign calls should not have negatives- there is nothing called negative calls. There are 2 possible reasons. 1. There were typos and errors made, a negative sign added to 10% of the dataset. 2. These should be 0. 
However, it is weird that the negative numbers have such a huge range (from -1 to -41). Hence, the first option seems to be more viable.

In [15]:
df["Age"].value_counts()

Age
150 years    4197
31 years     1747
32 years     1646
33 years     1643
36 years     1606
             ... 
92 years        4
89 years        2
91 years        2
98 years        2
95 years        1
Name: count, Length: 77, dtype: int64

Age of 150 years is not possible. 10% of data have 150 y.o., and removing it may cause huge info loss. I will be filling up with the mean Age to reduce as much info loss.

Summary of inspection:
1. No duplicates for the rows or client ID.
2. Housing Loan contains None (missing) data- I am assuming this means that no loan was taken and will be filling 'None' with 'no'.
3. Personal Loan contains None (missing) data- I am assuming this means that no loan was taken and will be filling 'None' with 'no'.
4. Age contains '150'- this is physically impossible. 10% of the overall data contains this error and removing it may cause important info to be lost. Hence, we will be filling it with the mean age.
5. Campaign calls should not be in the negatives - yet there are no 0s. There is also a big range of negative numbers, from -1 to -41. I am assuming these were typos made as only 10% of them were negative. They will be converted to posititve.
6. Previous Contact days having 999 days is normal- it means that they have not been contacted before. However, this means that I will need to be careful when scaling.
7. In contact method, there are 2 cell/cellular and 2 telephone/Telephone- they should be merged together so that there is only cellular and telephone.
8. Subscription status is our target.
9. Occupation, marital status, education level , housing and personal loan , contact methods and subscription status needs to be label encoded.

Filling up missing data: Housing Loan and Personal Loan.

In [16]:
df['Housing Loan'] = df['Housing Loan'].fillna('no')
print(df['Housing Loan'].value_counts(dropna=False)) # include NaN count to confirm no NaNs left

Housing Loan
no         32200
yes         8595
unknown      393
Name: count, dtype: int64


In [17]:
df['Personal Loan'] = df['Personal Loan'].fillna('no')
print(df['Personal Loan'].value_counts(dropna=False)) # include NaN count to confirm no NaNs left

Personal Loan
no         34678
yes         5633
unknown      877
Name: count, dtype: int64


Contact Method- Cellular and Telephone.

In [18]:
df['Contact Method'] = df['Contact Method'].replace(['Cell', 'cellular'], 'Cellular')
df['Contact Method'] = df['Contact Method'].replace(['Telephone', 'telephone'], 'Telephone')

print(df['Contact Method'].value_counts())

Contact Method
Cellular     26144
Telephone    15044
Name: count, dtype: int64


Filling Age

In [19]:
df['Age'] = df['Age'].str.replace('years', '', regex=False) #Let's remove the years so we can turn age into int.
df['Age'] = df['Age'].astype(int)
# Calculate the mean of 'Age' excluding values above 100
mean_age_excluding_outliers = df[df['Age'] <= 101]['Age'].mean()

# Replace ages above 100 with the calculated mean
df.loc[df['Age'] > 100, 'Age'] = mean_age_excluding_outliers

print(df['Age'].describe())
df_age_over_100_after = df[df['Age'] > 100]
print("Rows where 'Age' is greater than 100 after replacement:")
df_age_over_100_after #No more age 150.

count    41188.000000
mean        40.025303
std          9.890733
min         17.000000
25%         33.000000
50%         40.000000
75%         46.000000
max         98.000000
Name: Age, dtype: float64
Rows where 'Age' is greater than 100 after replacement:


C:\Users\Marcus\AppData\Local\Temp\ipykernel_2304\3922784848.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '40.025303452191075' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['Age'] > 100, 'Age'] = mean_age_excluding_outliers


,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status


In [20]:
df['Age'] = df['Age'].astype(int)
print(df['Age'].value_counts()) #int, no floats.

Age
40    5224
31    1747
32    1646
33    1643
36    1606
      ... 
92       4
89       2
91       2
98       2
95       1
Name: count, Length: 76, dtype: int64


Converting campaign calls to all positive.

In [21]:
df['Campaign Calls'] = df['Campaign Calls'].abs()
print("Any negatives left:", (df['Campaign Calls'] < 0).any())

Any negatives left: False


In [22]:
df

,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status
0,32885,57,technician,married,high.school,no,no,yes,Cellular,1,999,no
1,3170,55,unknown,married,unknown,unknown,yes,no,Telephone,2,999,no
2,32207,33,blue-collar,married,basic.9y,no,no,no,Cellular,1,999,no
3,9404,36,admin.,married,high.school,no,no,no,Telephone,4,999,no
4,14021,27,housemaid,married,high.school,no,no,no,Cellular,2,999,no
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,6266,58,retired,married,professional.course,unknown,no,no,Telephone,2,999,no
41184,11285,37,management,married,university.degree,no,no,no,Telephone,1,999,no
41185,38159,35,admin.,married,high.school,no,no,no,Cellular,1,4,yes
41186,861,40,management,married,university.degree,no,no,no,Telephone,2,999,no


## Feature Engineering
New column:


In [23]:
df['No_Prior_Contact'] = (df['Previous Contact Days'] == 999).astype(int)
print(df[['Previous Contact Days', 'No_Prior_Contact']].head(10))
print(df['No_Prior_Contact'].value_counts())

   Previous Contact Days  No_Prior_Contact
0                    999                 1
1                    999                 1
2                    999                 1
3                    999                 1
4                    999                 1
5                    999                 1
6                    999                 1
7                    999                 1
8                    999                 1
9                    999                 1
No_Prior_Contact
1    39673
0     1515
Name: count, dtype: int64


### Label Encoding:

In [24]:
from sklearn.preprocessing import LabelEncoder

label_cols = ['Occupation', 'Marital Status', 'Education Level',
              'Credit Default', 'Contact Method','Personal Loan', 'Housing Loan', 'Subscription Status']

le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col])

In [25]:
df

,Client ID,Age,Occupation,Marital Status,Education Level,Credit Default,Housing Loan,Personal Loan,Contact Method,Campaign Calls,Previous Contact Days,Subscription Status,No_Prior_Contact
0,32885,57,9,1,3,0,0,2,0,1,999,0,1
1,3170,55,11,1,7,1,2,0,1,2,999,0,1
2,32207,33,1,1,2,0,0,0,0,1,999,0,1
3,9404,36,0,1,3,0,0,0,1,4,999,0,1
4,14021,27,3,1,3,0,0,0,0,2,999,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,6266,58,5,1,5,1,0,0,1,2,999,0,1
41184,11285,37,4,1,6,0,0,0,1,1,999,0,1
41185,38159,35,0,1,3,0,0,0,0,1,4,1,0
41186,861,40,4,1,6,0,0,0,1,2,999,0,1


TESTTTTTTT